# NYC Felony Dataset Initial Stats and Plots

Source: https://catalog.data.gov/dataset/nypd-7-major-felony-incidents

---
### Initial Set Up

In [1]:
# Initial Imports
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
% matplotlib inline

ImportError: DLL load failed: %1 is not a valid Win32 application.

In [ ]:
# loading felony data set
felony_df = pd.read_csv("NYPD_7_Major_Felony_Incidents.csv", parse_dates = 'Occurrence Date')

In [ ]:
# reset index
felony_df.set_index('OBJECTID', inplace = True)

### Basic Dataset Info
__number of observations:__ 1123465  
__number of variables:__ 20  
__date range:__  1905-2015

In [ ]:
# num observations
print(len(felony_df))

In [ ]:
# print out date range
print(felony_df['Occurrence Year'].min())
print(felony_df['Occurrence Year'].max())

In [ ]:
# list out the column names and their data types
for col in felony_df.columns:
    print('{:25}{:20}'.format(col, str(felony_df[col].dtype)))

### Some Initial Plots.

In [ ]:
# plot crime incidence by borough (whole data set)
felony_df.Borough.value_counts().plot(kind = 'bar')
plt.title("Total Felonies by Borough (1905-2015)")

In [ ]:
# plot crime incidence by year 2000-2015)
counts = felony_df['Occurrence Year'].value_counts()
#counts.sort_values(inplace=True)
counts[list(range(2000,2016))].plot(kind = 'bar')
plt.title("Felonies committed in NYC by Year(2000-2015)")

### Starting to focus on Location.

In [ ]:
# Extract Lattitude and Longitude from the string coordinates
get_lat = lambda x: str(x).split(',')[0].replace('(','')
get_long = lambda x: str(x).split(',')[1].replace(')','')  
    
felony_df['Latitude'] = felony_df['Location 1'].map(get_lat)
felony_df['Longitude'] = felony_df['Location 1'].head().map(get_long)
felony_df['Latitude'].head()   
# for some reason' I'm getting an error when I try to convert the rest of the longitudes 
# I'm going to move on for now and deal with it later (maybe just change the loc data type?)